In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão de Python Neste Jupyter Notebook:', python_version())

# usaremos o filtro 'waning' para deixar mais limpo.
import warnings
warnings.filterwarnings('ignore')

### Representing Search Problems

In [ ]:
# searchProblem.py

In [ ]:
class Search_problem(object):
12 """A search problem consists of:
13 * a start node
14 * a neighbors function that gives the neighbors of a node
15 * a specification of a goal
16 * a (optional) heuristic function.
The methods must be overridden to define a search problem."""
18
19 def start_node(self):
20 """returns start node"""
21 raise NotImplementedError("start_node") # abstract method
22
23 def is_goal(self,node):
24 """is True if node is a goal"""
25 raise NotImplementedError("is_goal") # abstract method
26
27 def neighbors(self,node):
28 """returns a list of the arcs for the neighbors of node"""
29 raise NotImplementedError("neighbors") # abstract method
30
31 def heuristic(self,n):
32 """Gives the heuristic value of node n.
33 Returns 0 if not overridden."""
34 return 0

In [ ]:
36 class Arc(object):
37 """An arc has a from_node and a to_node node and a (non-negative)
cost"""
38 def __init__(self, from_node, to_node, cost=1, action=None):
39 assert cost >= 0, ("Cost cannot be negative for"+
40 str(from_node)+"->"+str(to_node)+", cost:
"+str(cost))
41 self.from_node = from_node
42 self.to_node = to_node
43 self.action = action
44 self.cost=cost
45
46 def __repr__(self):
47 """string representation of an arc"""
48 if self.action:
49 return str(self.from_node)+" --"+str(self.action)+"-->
"+str(self.to_node)
50 else:
51 return str(self.from_node)+" --> "+str(self.to_node)

In [ ]:
53 class Search_problem_from_explicit_graph(Search_problem):
54 """A search problem consists of:
55 * a list or set of nodes
56 * a list or set of arcs
57 * a start node
58 * a list or set of goal nodes
59 * a dictionary that maps each node into its heuristic value.
60 * a dictionary that maps each node into its (x,y) position
61 """
62
63 def __init__(self, nodes, arcs, start=None, goals=set(), hmap={},
positions={}):
64 self.neighs = {}
65 self.nodes = nodes
66 for node in nodes:
67 self.neighs[node]=[]
68 self.arcs = arcs
69 for arc in arcs:
70 self.neighs[arc.from_node].append(arc)
71 self.start = start
72 self.goals = goals
73 self.hmap = hmap
74 self.positions = positions
75
76 def start_node(self):
77 """returns start node"""
78 return self.start
79
80 def is_goal(self,node):
81 """is True if node is a goal"""
82 return node in self.goals
83
84 def neighbors(self,node):
85 """returns the neighbors of node"""
86 return self.neighs[node]
88 def heuristic(self,node):
89 """Gives the heuristic value of node n.
90 Returns 0 if not overridden in the hmap."""
91 if node in self.hmap:
92 return self.hmap[node]
93 else:
94 return 0
95
96 def __repr__(self):
97 """returns a string representation of the search problem"""
98 res=""
99 for arc in self.arcs:
100 res += str(arc)+". "
101 return res

In [ ]:
107 class Path(object):
108 """A path is either a node or a path followed by an arc"""
109
110 def __init__(self,initial,arc=None):
111 """initial is either a node (in which case arc is None) or
112 a path (in which case arc is an object of type Arc)"""
113 self.initial = initial
114 self.arc=arc
115 if arc is None:
116 self.cost=0
117 else:
118 self.cost = initial.cost+arc.cost
119
120 def end(self):
121 """returns the node at the end of the path"""
122 if self.arc is None:
123 return self.initial
124 else:
125 return self.arc.to_node
126
127 def nodes(self):
128 """enumerates the nodes for the path.
129 This starts at the end and enumerates nodes in the path
backwards."""
130 current = self
131 while current.arc is not None:
132 yield current.arc.to_node
133 current = current.initial
134 yield current.initial
135
136 def initial_nodes(self):
137 """enumerates the nodes for the path before the end node.
138 This starts at the end and enumerates nodes in the path
backwards."""
139 if self.arc is not None:
140 yield from self.initial.nodes()
141
142 def __repr__(self):
143 """returns a string representation of a path"""
144 if self.arc is None:
145 return str(self.initial)
146 elif self.arc.action:
147 return (str(self.initial)+"\n --"+str(self.arc.action)
148 +"--> "+str(self.arc.to_node))
149 else:
150 return str(self.initial)+" --> "+str(self.arc.to_node)

In [ ]:
problem1 = Search_problem_from_explicit_graph(
153 {'A','B','C','D','G'},
154 [Arc('A','B',3), Arc('A','C',1), Arc('B','D',1), Arc('B','G',3),
     155 Arc('C','B',1), Arc('C','D',3), Arc('D','G',1)],
156 start = 'A',
157 goals = {'G'},
158 positions={'A': (0, 2), 'B': (1, 1), 'C': (0,1), 'D': (1,0), 'G':
(2,0)})

In [ ]:
159 problem2 = Search_problem_from_explicit_graph(
160 {'a','b','c','d','e','g','h','j'},
161 [Arc('a','b',1), Arc('b','c',3), Arc('b','d',1), Arc('d','e',3),
162 Arc('d','g',1), Arc('a','h',3), Arc('h','j',1)],
163 start = 'a',
164 goals = {'g'},
165 positions={'a': (0, 0), 'b': (0, 1), 'c': (0,4), 'd': (1,1), 'e': (1,4),
166 'g': (2,1), 'h': (3,0), 'j': (3,1)})

In [ ]:
168 problem3 = Search_problem_from_explicit_graph(
169 {'a','b','c','d','e','g','h','j'},
170 [],
171 start = 'g',
172 goals = {'k','g'})

In [ ]:
174 simp_delivery_graph = Search_problem_from_explicit_graph(
175 {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'},
176 [ Arc('A', 'B', 2),
177 Arc('A', 'C', 3),
178 Arc('A', 'D', 4),
179 Arc('B', 'E', 2),
180 Arc('B', 'F', 3),
181 Arc('C', 'J', 7),
182 Arc('D', 'H', 4),
183 Arc('F', 'D', 2),
184 Arc('H', 'G', 3),
185 Arc('J', 'G', 4)],
186 start = 'A',
187 goals = {'G'},
188 hmap = {
189 'A': 7,
190 'B': 5,
191 'C': 9,
192 'D': 6,
193 'E': 3,
194 'F': 5,
195 'G': 0,
196 'H': 3,
197 'J': 4,
198 })

In [ ]:
199 cyclic_simp_delivery_graph = Search_problem_from_explicit_graph(
200 {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'},
201 [ Arc('A', 'B', 2),
202 Arc('A', 'C', 3),
203 Arc('A', 'D', 4),
204 Arc('B', 'A', 2),
205 Arc('B', 'E', 2),
206 Arc('B', 'F', 3),
207 Arc('C', 'A', 3),
208 Arc('C', 'J', 7),
209 Arc('D', 'A', 4),
210 Arc('D', 'H', 4),
211 Arc('F', 'B', 3),
212 Arc('F', 'D', 2),
213 Arc('G', 'H', 3),
214 Arc('G', 'J', 4),
215 Arc('H', 'D', 4),
216 Arc('H', 'G', 3),
217 Arc('J', 'C', 6),
218 Arc('J', 'G', 4)],
219 start = 'A',
220 goals = {'G'},
221 hmap = {
222 'A': 7,
223 'B': 5,
224 'C': 9,
225 'D': 6,
226 'E': 3,
227 'F': 5,
228 'G': 0,
229 'H': 3,
230 'J': 4,
231 })

In [ ]:
233 acyclic_delivery_problem = Search_problem_from_explicit_graph(
234 {'mail','ts','o103','o109','o111','b1','b2','b3','b4','c1','c2','c3',
235 'o125','o123','o119','r123','storage'},
236 [Arc('ts','mail',6),
237 Arc('o103','ts',8),
238 Arc('o103','b3',4),
239 Arc('o103','o109',12),
240 Arc('o109','o119',16),
241 Arc('o109','o111',4),
242 Arc('b1','c2',3),
243 Arc('b1','b2',6),
244 Arc('b2','b4',3),
245 Arc('b3','b1',4),
246 Arc('b3','b4',7),
247 Arc('b4','o109',7),
248 Arc('c1','c3',8),
249 Arc('c2','c3',6),
250 Arc('c2','c1',4),
251 Arc('o123','o125',4),
252 Arc('o123','r123',4),
253 Arc('o119','o123',9),
254 Arc('o119','storage',7)],
255 start = 'o103',
256 goals = {'r123'},
257 hmap = {
258 'mail' : 26,
259 'ts' : 23,
260 'o103' : 21,
261 'o109' : 24,
262 'o111' : 27,
263 'o119' : 11,
264 'o123' : 4,
265 'o125' : 6,
266 'r123' : 0,
267 'b1' : 13,
268 'b2' : 15,
269 'b3' : 17,
270 'b4' : 18,
271 'c1' : 6,
272 'c2' : 10,
273 'c3' : 12,
274 'storage' : 12
275 }
276 )

In [ ]:
278 cyclic_delivery_problem = Search_problem_from_explicit_graph(
279 {'mail','ts','o103','o109','o111','b1','b2','b3','b4','c1','c2','c3',
280 'o125','o123','o119','r123','storage'},
281 [ Arc('ts','mail',6), Arc('mail','ts',6),
282 Arc('o103','ts',8), Arc('ts','o103',8),
283 Arc('o103','b3',4),
284 Arc('o103','o109',12), Arc('o109','o103',12),
285 Arc('o109','o119',16), Arc('o119','o109',16),
286 Arc('o109','o111',4), Arc('o111','o109',4),
287 Arc('b1','c2',3),
288 Arc('b1','b2',6), Arc('b2','b1',6),
289 Arc('b2','b4',3), Arc('b4','b2',3),
290 Arc('b3','b1',4), Arc('b1','b3',4),
291 Arc('b3','b4',7), Arc('b4','b3',7),
292 Arc('b4','o109',7),
293 Arc('c1','c3',8), Arc('c3','c1',8),
294 Arc('c2','c3',6), Arc('c3','c2',6),
295 Arc('c2','c1',4), Arc('c1','c2',4),
296 Arc('o123','o125',4), Arc('o125','o123',4),
297 Arc('o123','r123',4), Arc('r123','o123',4),
298 Arc('o119','o123',9), Arc('o123','o119',9),
299 Arc('o119','storage',7), Arc('storage','o119',7)],
300 start = 'o103',
301 goals = {'r123'},
302 hmap = {
303 'mail' : 26,
304 'ts' : 23,
305 'o103' : 21,
306 'o109' : 24,
307 'o111' : 27,
308 'o119' : 11,
309 'o123' : 4,
310 'o125' : 6,
311 'r123' : 0,
312 'b1' : 13,
313 'b2' : 15,
314 'b3' : 17,
315 'b4' : 18,
316 'c1' : 6,
317 'c2' : 10,
318 'c3' : 12,
319 'storage' : 12
320 }
321 )

In [ ]:
%reload_ext watermark
%watermark -a "Caique Miranda" -gu "caiquemiranda" -iv

### End.